In [35]:
# file description : data -> class별로 분류되어있음
# disease 폴더 : class 7, 8 섞여 있음 -> data폴더로 옮겨서 train test val 나누기

import os 
import shutil
import cv2

In [36]:
# 현재 디렉토리 확인
print(os.getcwd())

/Users/bykwon/Desktop/dl/finalproj


In [37]:
path = os.getcwd() + '/disease'

# disease 폴더 내의 파일들을 data 폴더로 옮기기
file_list = os.listdir(path)

for file in file_list:
    file_name_list = file.split("_")
    if file_name_list[3] == '07':
        shutil.move(path + '/' + file, os.getcwd() + '/data/class7')
    elif file_name_list[3] == '08':
        shutil.move(path + '/' + file, os.getcwd() + '/data/class8')

In [38]:
# json -> txt format
import json

def convert_to_yolo(filename):
    json_data = json.load(open(filename))
    class_label = json_data["annotations"]["disease"]
    image_width = json_data["description"]["width"]
    image_height = json_data["description"]["height"]
    xtl = json_data["annotations"]["points"][0]["xtl"]
    ytl = json_data["annotations"]["points"][0]["ytl"]
    xbr = json_data["annotations"]["points"][0]["xbr"]
    ybr = json_data["annotations"]["points"][0]["ybr"]

    if class_label == 7:
        class_label = 1
    elif class_label == 8:
        class_label = 2

    # YOLO format calculations
    center_x = (xtl + xbr) / (2 * image_width)
    center_y = (ytl + ybr) / (2 * image_height)
    width = (xbr - xtl) / image_width
    height = (ybr - ytl) / image_height

    with open(filename.replace(".json", ".txt"), "w") as f:
        f.write(f"{class_label} {center_x} {center_y} {width} {height}")

    # return yolo_format

In [39]:
# 현재 디렉토리 확인
print(os.getcwd())

/Users/bykwon/Desktop/dl/finalproj


In [40]:
path_list = [os.getcwd() + '/data/class0', os.getcwd() + '/data/class7', os.getcwd() + '/data/class8']

for path in path_list:
    for file in os.listdir(path):
        if file.endswith(".json"):
            convert_to_yolo(path + '/' + file)

In [41]:
# json file 밖으로 꺼내기

# 현재 디렉토리 확인
print(os.getcwd())

/Users/bykwon/Desktop/dl/finalproj


In [42]:
for path in path_list:
    for file in os.listdir(path):
        if file.endswith(".json"):
            shutil.move(path + '/' + file, os.getcwd() + '/disease')

In [43]:
# train, test, val 나누기
import splitfolders

splitfolders.ratio("data", output="dataset", seed=921, ratio=(.8, .1, .1), group_prefix=2)

Copying files: 404 files [00:01, 320.80 files/s]


In [44]:
# 각 폴더마다 images / labels 폴더 생성
import os
print(os.getcwd())

/Users/bykwon/Desktop/dl/finalproj


In [45]:
path_list = [os.getcwd() + '/dataset/train', os.getcwd() + '/dataset/test', os.getcwd() + '/dataset/val']

for path in path_list:
    for folder in os.listdir(path):
        os.mkdir(path + '/' + folder + '/images')
        os.mkdir(path + '/' + folder + '/labels')

        # 항목 이동
        for file in os.listdir(path + '/' + folder):
            if file.endswith(".jpg") or file.endswith(".JPG"):
                shutil.move(path + '/' + folder + '/' + file, path + '/' + folder + '/images')
            elif file.endswith(".txt"):
                shutil.move(path + '/' + folder + '/' + file, path + '/' + folder + '/labels')

In [46]:
new_extension = '.txt'
for path in path_list:
    for folder in os.listdir(path):
        for file in os.listdir(path + '/' + folder + '/labels'):
            if file.endswith(".txt"):
                # 이름 변경하기
                if file.endswith('.jpg.txt'):
                    os.rename(path + '/' + folder + '/labels/' + file, path + '/' + folder + '/labels/' + file.replace('.jpg.txt', new_extension))
                elif file.endswith('.JPG.txt'):
                    os.rename(path + '/' + folder + '/labels/' + file, path + '/' + folder + '/labels/' + file.replace('.JPG.txt', new_extension))